In [ ]:
%load_ext autoreload
%autoreload 2
from visualize import (
    shot_frames_navigator,
    animate_pitch,
    animate_chain,
    manual_labeling,
)
from data import load_and_process_soccer_events, load_and_process_metadata
from label import pos_labeling
from IPython.display import HTML
import polars as pl

In [ ]:
event_df, players_df = load_and_process_soccer_events("/home/soccerdata/FIFA_WorldCup_2022/Event Data")
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

In [ ]:
players_df

In [ ]:
players_df.null_count()

In [ ]:
players_df.filter(pl.col('velocity').is_null() & pl.col('team').is_not_null() & ~pl.col('gameEventId').is_in([6729865, 6528353, 6499785, 6498876]))

In [ ]:
pos_chains = pos_labeling(event_df, 3)
pos_chains_flattened = [x for xs in pos_chains for x in xs]

In [ ]:
event_df_filtered = event_df.filter(pl.col('index').is_in(pos_chains_flattened))
players_df_filtered = players_df.join(event_df_filtered, on='gameEventId')

In [ ]:
players_df_filtered.null_count()

In [ ]:
players_df_filtered

In [ ]:
players_df_filtered.filter(pl.col('velocity').is_nan())

In [ ]:
players_df_filtered.filter(pl.col('velocity') > 20)

In [ ]:
shot_frames_navigator(pos_chains[31], event_df, players_df, "/home/soccerdata/frames",show_video=True)

In [ ]:
anim = animate_pitch(pos_chains[1400], event_df, players_df, metadata_df, interval=500)
HTML(anim.to_jshtml())

In [ ]:
manual_labels = manual_labeling(
    (0, 2), animate_chain, pos_chains, event_df, players_df, metadata_df
)